In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import sys
import os
import numpy as np
from easydict import EasyDict as edict
from glob import glob
from tqdm import tqdm

sys.path.append('../insightface/src/align')
sys.path.append('../insightface/src/common')
sys.path.append('../insightface/src/megaface')

import detect_face
import cv2

import mxnet as mx
import face_preprocess

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Load model

In [2]:
nets = []
image_shape = [3,112,112]
models_list = ['/mnt/ssd2/David/faces/insightface/recognition/models/r100-arcface-emore/model,1']

for model in models_list:
    vec = model.split(',')
    assert len(vec)>1
    prefix = vec[0]
    epoch = int(vec[1])
    print('loading',prefix, epoch)
    net = edict()
    net.ctx = mx.gpu(0)
    net.sym, net.arg_params, net.aux_params = mx.model.load_checkpoint(prefix, epoch)
    
    all_layers = net.sym.get_internals()
    
    net.sym = all_layers['fc1_output']
    net.model = mx.mod.Module(symbol=net.sym, context=net.ctx, label_names = None)
    net.model.bind(data_shapes=[('data', (1, 3, image_shape[1], image_shape[2]))])
    net.model.set_params(net.arg_params, net.aux_params)
    nets.append(net)

loading /mnt/ssd2/David/faces/insightface/recognition/models/r100-arcface-emore/model 1


In [3]:
def get_feature(image_list, net):
    images = []
    for image_path in image_list:
        img = face_preprocess.read_image(image_path, mode='rgb')
        img = np.transpose( img, (2,0,1) )
        images.append(img)

    input_blob = np.array(images)
    data = mx.nd.array(input_blob)
    db = mx.io.DataBatch(data=(data,))
    net.model.forward(db, is_train=False)
    embedding = net.model.get_outputs()[0].asnumpy()
        
    return embedding

### Process images

In [7]:
images_folder = "/mnt/ssd2/Datasets/faces_emore_images"
dst_folder = "/mnt/ssd2/Datasets/faces_emore_labels"
images = glob(os.path.join(images_folder, "*.png"))
np.random.shuffle(images)
print("Images:", len(images))

Images: 5822653


In [10]:
batch_size = 50
files = []

for filename in tqdm(images):
    files.append(filename)
        
    if len(files) == batch_size:
        vectors = get_feature(files, nets[0])
        for path, vector in zip(files, vectors):
            np.save(path.replace(images_folder, dst_folder)+"_fc1", vector)
        files = []

vectors = get_feature(files, nets[0])
for path, vector in zip(files, vectors):
    np.save(path.replace(images_folder, dst_folder)+"_fc1", vector)
files = []

100%|██████████| 5822653/5822653 [5:36:41<00:00, 288.23it/s]  


In [ ]:
print("finish")